In [1]:
import os
import pandas as pd

DATA_DIR = os.path.join('..', 'data', 'raw', 'reviews', '1pct_10pct')

In [2]:
genre_df = (pd.read_csv(os.path.join(DATA_DIR, file))
            for file in os.listdir(DATA_DIR))

reviews = pd.concat(genre_df, ignore_index=True)

In [3]:
reviews.head()

,id,text,rating,date,title,author,helpfulness
0,/title/tt0468569/,Best movie ever. Heath ledger's work is phenom...,10.0,12 January 2021,Perfect combo\n,/user/ur95396995/?ref_=tt_urv,\n 171 out of 185 found thi...
1,/title/tt0468569/,Totally one of the greatest movie titles ever ...,10.0,9 January 2021,The Dark Knight\n,/user/ur109215140/?ref_=tt_urv,\n 144 out of 158 found thi...
2,/title/tt0468569/,This movie is a work of art. The finest sequel...,10.0,17 February 2021,This town deserves a better class of criminal!\n,/user/ur129557514/?ref_=tt_urv,\n 50 out of 54 found this ...
3,/title/tt0468569/,"Confidently directed, dark, brooding, and pack...",10.0,12 February 2020,The Dark Knight\n,/user/ur87850731/?ref_=tt_urv,\n 404 out of 471 found thi...
4,/title/tt0468569/,It is just what you want for the best movie. G...,10.0,7 October 2019,MASTERPIECE\n,/user/ur108519953/?ref_=tt_urv,\n 217 out of 251 found thi...


In [30]:
import pandas as pd
data_path = '../data/raw/details/1pct/action.csv'
action = pd.read_csv(data_path)
action.head()

,original_title,review_summary,agg_rating,actors,imdb_recommendations,storyline,tagline,certificate,details,boxoffice,techspecs,title_id
0,Original title: The Dark Knight,"{'n_user_reviews': '7.7KUser reviews', 'n_crit...",9.0/102.5M,{'Christian Bale': '/name/nm0000288?ref_=tt_cl...,"['/title/tt1345836/?ref_=tt_sims_tt_t_1', '/ti...",Set within a year after the events of Batman B...,TaglinesWhy So Serious?,Certificate14+,"Release dateAugust 14, 2008 (Russia)Countries ...","Budget$185,000,000 (estimated)Gross US & Canad...",Runtime2 hours 32 minutesSound mixDolby Digita...,/title/tt0468569/
1,Original title: Inception,"{'n_user_reviews': '4.4KUser reviews', 'n_crit...",8.8/102.2M,{'Leonardo DiCaprio': '/name/nm0000138?ref_=tt...,"['/title/tt0816692/?ref_=tt_sims_tt_t_1', '/ti...","Dom Cobb is a skilled thief, the absolute best...",TaglinesYour mind is the scene of the crime,Certificate12+,"Release dateJuly 22, 2010 (Russia)Countries of...","Budget$160,000,000 (estimated)Gross US & Canad...",Runtime2 hours 28 minutesColorColorSound mixDo...,/title/tt1375666/
2,Original title: The Matrix,"{'n_user_reviews': '4.6KUser reviews', 'n_crit...",8.7/101.8M,{'Keanu Reeves': '/name/nm0000206?ref_=tt_cl_t...,"['/title/tt1375666/?ref_=tt_sims_tt_t_1', '/ti...",Thomas A. Anderson is a man living two lives. ...,TaglinesFree your mind,Certificate16+,"Release dateOctober 14, 1999 (Russia)Countries...","Budget$63,000,000 (estimated)Gross US & Canada...",Runtime2 hours 16 minutesColorColorSound mixDo...,/title/tt0133093/
3,Original title: The Lord of the Rings: The Fel...,"{'n_user_reviews': '5.5KUser reviews', 'n_crit...",8.8/101.7M,{'Elijah Wood': '/name/nm0000704?ref_=tt_cl_t_...,"['/title/tt0167261/?ref_=tt_sims_tt_t_1', '/ti...",An ancient Ring thought lost for centuries has...,TaglinesThe Legend Comes to Life,Certificate12+,"Release dateMarch 1, 2002 (Russia)Countries of...","Budget$93,000,000 (estimated)Gross US & Canada...",Runtime2 hours 58 minutesColorColorSound mixDT...,/title/tt0120737/
4,Original title: The Lord of the Rings: The Ret...,"{'n_user_reviews': '3.9KUser reviews', 'n_crit...",8.9/101.7M,{'Elijah Wood': '/name/nm0000704?ref_=tt_cl_t_...,"['/title/tt0167261/?ref_=tt_sims_tt_t_1', '/ti...",The final confrontation between the forces of ...,TaglinesThe eye of the enemy is moving.,Certificate12+,"Release dateJanuary 22, 2004 (Russia)Countries...","Budget$94,000,000 (estimated)Gross US & Canada...",Runtime3 hours 21 minutesColorColorSound mixDT...,/title/tt0167260/


In [22]:
# separator = (
#     'Release date'
#     '|Countries of origin'
#     '|Country of origin'
#     '|Official sites'
#     '|Official site'
#     '|Languages'
#     '|Language'
#     '|Also known as'
#     '|Filming locations'
#     '|Filming location'
#     '|Production companies'
#     '|Production company'
# )
# separator = (
#     'Release date'
#     '|Country of origin'
# )
# sp = action['details'].str.split(separator, expand=True)
# sp.head(100)

In [10]:
# import pandas as pd
# from recsys.utils import load_obj
# genre_ids = os.listdir('../data/raw/identifiers/1pct')
# ids = {}
# for genre_ in genre_ids:
#     genre_loc = '../data/raw/identifiers/1pct/' + genre_
#     ids[genre_.split('.pkl')[0]] = load_obj(genre_loc)

# details_genres = os.listdir('../data/raw/details/1pct/')
# for genre in details_genres:
#     if genre == 'posters':
#         continue
#     genre_name = genre.split(".csv")[0]
#     data_path = f'../data/raw/details/1pct/{genre_name}.csv'
#     data = pd.read_csv(data_path)
#     if genre_name == 'drama':
#         data['title_id'] = ids[genre_name][1040:]
#     else:
#         data['title_id'] = ids[genre_name]
#     cols_without_unnamed = [col for col in data.columns if 'Unnamed:' not in col]
#     data[cols_without_unnamed].to_csv(data_path, index=False)
#     print(genre)
# data_path = '../data/raw/details/1pct/action.csv'
# action = pd.read_csv(data_path)
# action.head()

# action['title_id'] = ids['action']
# action

action.csv
adventure.csv
animation.csv
biography.csv
comedy.csv
crime.csv
drama.csv
family.csv
fantasy.csv
film-noir.csv
history.csv
horror.csv
music.csv
musical.csv
mystery.csv
romance.csv
sci-fi.csv
sport.csv
thriller.csv
war.csv
western.csv


In [10]:
import pandas as pd
from recsys.core.data import CSVDataLoader
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [158]:
data_path = '../data/raw/details/1pct/action.csv'
loader = CSVDataLoader(data_path)

In [165]:
from recsys.core.etl import RawReviewsTransformer, RawDetailsTransformer

In [166]:
reviews_processor = RawDetailsProcessor(loader)

In [167]:
a, b, c = reviews_processor.process()

In [170]:
c

,title_id,suggested_title_id,order_num
0,/title/tt0468569/,/title/tt1345836/,1
1,/title/tt0468569/,/title/tt1375666/,2
2,/title/tt0468569/,/title/tt0372784/,3
3,/title/tt0468569/,/title/tt0111161/,4
4,/title/tt0468569/,/title/tt0816692/,5
...,...,...,...
5995,/title/tt2404463/,/title/tt0212346/,8
5996,/title/tt2404463/,/title/tt1289401/,9
5997,/title/tt2404463/,/title/tt4651520/,10
5998,/title/tt2404463/,/title/tt0385307/,11
